*You may navigate the notebook through the table of contents on the left.*

---
# **Dimensionality reduction for neural data**
---

Leonardo Agueci <sup>1</sup>, Arthur Pellegrino<sup>2</sup> and N. Alex Cayco Gajic<sup>1</sup>

<sup>1</sup>Ecole Normale Superieure Paris, <sup>2</sup>University of Edinburgh

# Introduction

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1bWAEsI-pGOSUWmMW3TfSmTVk1B32jSkR" width="550px;">
</p>

In this tutorial you will extensively learn about principal component analysis (PCA) [1,2] and its application on neural data. 
- In __Part I__, we introduce you to the theory underlying this dimensionality reduction method, and its relationship with singular value decomposition (SVD). Then, we will extend its usage to tensors, basic step for applying this method to standard neural data; as a ___bonus___, you will find a brief introduction to tensor component analysis (TCA) and to sliceTCA. Finally, you will find a description of the bi-cross validation method. 
- In __Part II__ you will use this theoretical background to implement PCA.
- __Part III__ focuses on how to cross-validate the number of components.
- In __Part IV__ you will apply PCA to neural data recorded during hand movement.
- Finally, __Part V__ we will finally try to gain some understanding of what could be encoded in our neural recordings using some simple linear decoders

___Remarks:___

- Code cells are partially filled sometimes, for helping you with coding and visualization. Although you can change any part of the code as you prefer (e.g. for improving visualization), we recommend to add you code only in the parts delimited as follows (for plots, you might find some settings that might be helpful):

```(python)
# Fill in your code below
###########################

# Instruction 1 
# your code here

# Instruction 2
plt.plot(# your code here , some_useful_settings)

###########################
```

- __If you want to run this notebook offline, make sure your python environment has installed the following libraries:__
    - matplotlib
    - numpy
    - pickle
    - mpl_toolkits
    - sklearn
    - tqdm
    - scipy


# **Part I.** The theory behind (multi)linear dimensionality reduction



One of the reasons for the enduring popularity of PCA is its analytical interpretability. To demonstrate this, we will present two mathematically equivalent formulations of PCA: 1) identifying the directions of maximum projected variance, and 2) finding the best low-dimensional approximation to the data matrix. By setting up these two questions as constrained optimization problems (following [2]), it can be shown that the two formulations yield the same solution: the eigenvalues and eigenvectors of the data covariance matrix. We will then cover the relationship between PCA and singular value decomposition (SVD), and the Eckart-Young Theorem which guarantees that the SVD is the closest low-rank approximation to the data matrix. The equivalence of these two formulations (the closest low rank approximation and vs directions of maximum variance) will set up the intuition necessary to understand data tensor extensions such as TCA or sliceTCA.

## The equivalent formulations of PCA

### PCA as the directions of maximum projected variance

When discussing PCA, we follow convention that we are working with a data matrix $\mathbf{X} \in \mathbb{R}^{N \times K}$ where $N$ is the number of neurons and $K$ the number of samples. We further assume that the data has already been centered, that is the activity of each neuron has been subtracted its mean. The sample covariance matrix $\mathbf{\hat\Sigma}\in \mathbb{R}^{N \times N}$ is then defined as $\mathbf{\hat \Sigma} = \frac{1}{N_\text{samples}}\mathbf{X}\mathbf{X}^T$.

We start by seeking the vector $\mathbf{u} \in \mathbb{R}^N$ that maximizes the variance of the projected data. We further add the constraint that $\bf u$ must be normalized ($\mathbf{u}^T\mathbf{u}=1$, else the problem is unbounded). Then, the projected data is simply $\mathbf{u}^T\mathbf{X}\in \mathbb{R}^{K}$. The projected variance is then given by:

\begin{align}
\text{var}(\mathbf{u}^T\mathbf{X}) =& \frac{1}{N_\text{samples}}(\mathbf{u}^T \mathbf{X})(\mathbf{X}^T \mathbf{u}) \\
=&\mathbf{u}^T \left ( \frac{1}{N_\text{samples}}\mathbf{X}\mathbf{X}^T \right ) \mathbf{u} \\
=& \mathbf{u}^T \mathbf{\hat \Sigma} \mathbf{u}.
\end{align}

One way to solve such an optimization problem is to use [Lagrange multipliers](https://en.wikipedia.org/wiki/Lagrange_multiplier). First, we define the Lagrange function:

$$ L(\mathbf{u},\lambda) = \mathbf{u}^T \mathbf{\hat \Sigma u} + \lambda(1-\mathbf{u}^T\mathbf{u})$$

A necessary condition for $\mathbf{u}$ to be a maximum under the imposed constraint is that it must be a zero of the gradient of the Lagrange function:

$$ \nabla_{\mathbf u} L = \mathbf{\hat \Sigma u} - \lambda \mathbf{u} = 0 $$

That is, $ \mathbf{\hat \Sigma u} = \lambda \mathbf{u}$, therefore  $\mathbf{u}$ must be an eigenvector of the data covariance matrix and $\lambda$ its corresponding eigenvalue. Combining this with the projected variance as derived above, we get $\text{var}(\mathbf{u}^T\mathbf{X}) = \mathbf{u}^T \mathbf{\hat \Sigma} \mathbf{u} = \lambda\mathbf{u}^T  \mathbf{u} = \lambda$. Therefore the vector that maximizes the projected variance is the eigenvector of $\bf \hat \Sigma$ with the largest eigenvalue.

We can show inductively that by repeating this argument (with an additional constraint that each vector is orthogonal to the previously identified vectors), that the directions of maximum projected variance are given by the eigenvectors of $\bf \hat \Sigma$ and the projected variances by their corresponding eigenvalues.

The first $R$ principal components are given by projecting the data $\bf X$ onto the $R$ eigenvectors of the covariance matrix with largest eigenvalues. The eigenvectors are often called the loadings or weights, and the projected data are often called the scores.

Finally, we note that since $\bf \hat \Sigma$ is symmetric, its eigenvectors form an orthonormal basis for $\mathbb{R}^N$. In the context of PCA, this means that the loading vectors are orthogonal and the scores are uncorrelated.

### PCA as the low-dimensional projection that minimizes reconstruction error

In this second formulation we will attempt to find the low-dimensional projection of $\mathbf{X}$ with the smallest *reconstruction error*. We'll denote each sample (i.e. column) of the data matrix as ${\bf x}_n\in\mathbb{R}^N$. We thus seek an $R$-dimensional subspace such that, on average over all samples, the data projected on this subspace $\mathbf{\hat x}_n\in\mathbb{R}^N$ is closest to ${\bf x}_n$. We'll call our candidate orthonormal basis $\mathbf{u}_1,\dots,\mathbf{u}_R\in\mathbb{R}^N$, which spans this $R$-dimensional subspace within the $N$-dimensional activity space. Then, the estimate of each sample can be written as the linear combination:

$${\bf \hat x}_n =\sum_{i=1}^R \beta_{ni} \mathbf{u}_i$$

for some coefficients $\beta_{ni}$, which we would like to find as a function of $\mathbf{u}_i$ and $\mathbf{x}_n$.

Note that we're still missing $N-R$ vectors needed to form a complete orthonormal basis for the full activity space: we'll simply call such vectors $\mathbf{u}_{R+1},\dots,\mathbf{u}_N$. This means that we can rewrite each sample as:

$${\bf x}_n =\sum_{i=1}^N \alpha_{ni} \mathbf{u}_i$$

Since the $\mathbf{u}_i$'s are orthonormal, the coefficients are simply given by the projections of the data onto each basis vector: $\alpha_{ni} =\mathbf{x}_n^T \mathbf{u}_{i} $. You can verify this by taking the dot product of the above equation with $\mathbf{u}_j$ (remembering that $\mathbf{u}_i$'s being orthonormal is defined as: $\mathbf{u}_i\cdot \mathbf{u}_j=1$ if $i=j$ else $0$).

We can write the mean-squared error as:
$$ E =  \frac{1}{N \cdot N_\text{samples}} \sum_{n=1}^{N_\text{samples}} \lVert \mathbf{x}_n - \mathbf{\hat x}_n \rVert^2 $$
First, using our expressions for the samples above, we can write the residual on the right as:

\begin{align}
 \mathbf{x}_n - \mathbf{\hat x}_n = \sum_{i=1}^R (\alpha_{ni}-\beta_{ni})\mathbf{u}_i + \sum_{j=R+1}^N \alpha_{nj}\mathbf{u}_j
\end{align}

Now, noting that $\lVert \mathbf{x}_n - \mathbf{\hat x}_n \rVert^2 = ( \mathbf{x}_n - \mathbf{\hat x}_n )\cdot  (\mathbf{x}_n - \mathbf{\hat x}_n)$, and using the fact that the $\mathbf{u}_i$'s are orthonormal, the norm of the residual can be simplified as:

\begin{align}
\lVert \mathbf{x}_n - \mathbf{\hat x}_n \rVert^2 = \sum_{i=1}^R (\alpha_{ni}-\beta_{ni})^2 + \sum_{j=R+1}^N \alpha_{nj}^2
\end{align}

Finally, we can minimize the error by differentiating it as:
$$\frac{d E}{d \beta_{ni}} =  \frac{2}{N \cdot N_\text{samples}} (\beta_{ni} - \alpha_{ni})$$

which means that the $\beta_{ni}$ that minimizes the MSE is given simply by $\beta_{ni} = \alpha_{ni} = \mathbf{x}_n^T \mathbf{u}_i$. In retrospect this makes sense since the error should be minimized by projecting the data onto the low-dimensional subspace. We can now expand the error as:

\begin{align}
E =& \frac{1}{N \cdot N_\text{samples}} \sum_{n=1}^{N_\text{samples}} \sum_{j=R+1}^N \alpha_{nj}^2 \\
=& \frac{1}{N \cdot N_\text{samples}} \sum_{n=1}^{N_\text{samples}} \sum_{j=R+1}^N (\mathbf{x}_n^T \mathbf{u}_j)^2 \\
=& \frac{1}{N \cdot N_\text{samples}} \sum_{n=1}^{N_\text{samples}} \sum_{j=R+1}^N (\mathbf{u}_j^T\mathbf{x}_n )(\mathbf{x}_n^T \mathbf{u}_j) \\
=& \frac{1}{N} \sum_{j=R+1}^N \mathbf{u}_j^T \mathbf{\hat \Sigma u}_j
\end{align}
Where last step follows from $\mathbf{x}_n$ being mean-centered. It now remains to find an orthonormal basis $\mathbf{u}_j$ that minimizes the mean squared error. In the case that $R=N-1$, we have the same Lagrangian as before (up to a scaling factor):
$$ L(\mathbf{u},\lambda) = \frac{1}{N} \mathbf{u}^T \mathbf{\hat \Sigma u} + \lambda(1-\mathbf{u}^T\mathbf{u})$$
Again the solutions are the eigenvectors of the covariance matrix. This time, since we want to *minimize* the reconstruction error, we choose the eigenvector with smallest eigenvalue for the residual, which means that the low-dimensional subspace we project the data on is spanned by the remaining eigenvectors with larger eigenvalues.

*In general,* it can be shown that taking the $R$ largest eivenvectors of the covariance matrix as the principal subspace solves the constrained minimization of the MSE, resulting in the same solution as in the previous section. *Therefore,* PCA can be formulated in two mathematically equivalent forms: maximizing the directions of projected variance, or minimizing the reconstruction error of a low-dimensional projection.


### PCA as a low-rank approximation to the data matrix

So far we saw PCA as solving two different constrained optimization problems by projecting of the data onto the eigenvectors of the covariance matrix. Further, since the covariance matrix has an orthonormal set of eigenvectors, this projection can be considered as a change of basis from Cartesian coordinates (i.e., each dimension represents a different neuron) to a more "natural" basis of the data (i.e., each dimension represents a different covariability pattern across the population). Mathematically this projection is given by the equation:
$$ \mathbf{Z} = \mathbf{U}^T \mathbf{X}$$
where $\bf U \in \mathbb{R}^{N\times N}$ is a matrix whose columns are given by the orthonormal eigenvectors of $\mathbf{\hat \Sigma}$, in order of descending eigenvalue magnitude (which is the variance of the projected data, therefore this ranks the components by how much variance of the data they capture).

Note that since its columns $\mathbf{u}_i$ are orthonormal, $\bf U$ is a unitary matrix. An important property of unitary matrices is that $\mathbf{U}^{-1} = \mathbf{U}^T$. To see this, note that the $\mathbf{U}^T \mathbf{U} = \mathbf{I}$ falls out naturally from matrix multiplication, as $(\mathbf{U}^T \mathbf{U})_{ij}=\mathbf{u}_i^T\mathbf{u}_j = \delta_{ij}$ by orthonormality. To prove that $ \mathbf{U} \mathbf{U}^T = \mathbf{I}$, we can multiply both sides of the equation above by $\mathbf{U}$ on the left and $\mathbf{U}^{-1}$ on the right as follows:
\begin{align}
\mathbf{U}^T \mathbf{U} &= \mathbf{I} \\
\mathbf{U}(\mathbf{U}^T \mathbf{U})\mathbf{U}^{-1} &= \mathbf{U}\mathbf{U}^{-1}\\
\mathbf{U}\mathbf{U}^T &= \mathbf{I} \\
\end{align}

This is important in the context of PCA because it means we can easily move $\mathbf{U}$ to the other side of the equation to project the scores back into the original basis, returning the original data matrix:
$$\mathbf{U} \mathbf{Z} = \mathbf{X}$$
and now we see that PCA is a *matrix decomposition*, meaning that it decomposes the data matrix into a product of two matrices: the new basis vectors (or loadings) $\bf U$ and the projected data (or scores) $\mathbf Z$.

In dimensionality reduction, the purpose is to reduce the number of features used to capture the data. In PCA, this is done by taking the first $R$ components, which correspond to the largest eigenvalues (and thus the directions of largest projected variance). To do this we simply truncate the projection matrix after the first $R$ columns to get $\mathbf{ U }_{R}\in \mathbb{R}^{N\times R}$. Then the $R$-dimensional scores are still given by the same formula:
$$ \mathbf{Z}_R = \mathbf{U}_{R}^T \mathbf{X}$$

However we cannot simply move $\mathbf{U}_{R}$ to the other side of the equation. While we still have $\mathbf{U}_{R}^T  \mathbf{U}_{R} = \mathbf{I}$ (due to orthonormality of the columns), we can no longer guarantee that $ \mathbf{U}_{R} \mathbf{U}_{R}^T  = \mathbf{I}$ since our proof above relied on the existence of $\mathbf{U}_{R}^{-1}$, which does not exist for non-square matrices ($R< N$)! Geometrically, this makes sense since we can project from a high-dimensional space to a low-dimensional space but cannot invert the projection to recover the exact original data.

We can try to revert the transformation anyway by projecting the top $R$ components back into the $N$-dimensional space using $\mathbf{U}_R$. This results in the following *approximation* to the data matrix:
$$ \mathbf{ \hat X} = \mathbf{U}_{R} \mathbf{Z}_R$$
Another way to write this approxmination is to expand it as a sum of the outer products of the column.
$$ \mathbf{ \hat X} = \sum_{r=1}^R \mathbf{u}_{r} \mathbf{ z}^T_r$$
where $ \mathbf{u}_{r}$ are the columns of $ \mathbf{U}$ and $ \mathbf{ z}^T_r$ are the rows of $\mathbf{Z}$. Each column of $\mathbf{\hat X}$ is a linear combination of the $R$ basis vectors, so the rank of $ \mathbf{ \hat X} $ is (at most) $R$. Equivalently, the approximated data will lie on an $R$-dimensional subspace of $\mathbb{R}^N$ spanned by the new basis $\mathbf{u}_1,\dots,\mathbf{u}_R$.

This brings us back to the low-dimensional projection view that we saw previously, which further suggests that it may be the best low-rank approximation of the data matrix. This is indeed the case, as we will see in the next section.


### Relationship between PCA and SVD

PCA is very closely related to a more general matrix decomposition called the singular value decomposition (SVD). It is an amazing fact that any rank-$R$ matrix can be decomposed into three matrices:
$$ {\mathbf X} = {\mathbf U S} \mathbf{V}^T$$
where ${\mathbf U} \in \mathbb{R}^{N \times R}$ and $\mathbf{V} \in \mathbb{R}^{K \times R}$ have orthonormal columns (called the left and right singular vectors, respectively) that define new rotated bases for their respective vector spaces, and $\mathbf{S}\in \mathbb{R}^{R \times R}$ is a positive diagonal matrix (whose diagonal elements $\sigma_i$ are called the singular values, and are ranked in decreasing order). Most often, you will equivalently find it defined with, $U\in\mathbb{R}^{N\times \min\{N,K\}}$, $S\in\mathbb{R}^{\min \{N,K\}\times \min\{N,K\}}$, $V\in\mathbb{R}^{\min\{N,K\}\times N}$, in which case the rank of $\bf X$ is simply the number of non-zero singular values. The SVD can be thought of as decomposing a matrix into how it operates on vectors: by taking a $K$-dimensional vector, rotating it into a new basis, shrinking or stretching the new coordinates according to $\sigma_i$, and projecting it into $\mathbb{R}^N$.

How does this relate to PCA? To see this, let's again assume that $\mathbf{X}$ is mean-centered and calculate the covariance matrix using the SVD:
\begin{align}
\mathbf{\hat \Sigma} =& \frac{1}{N_\text{samples}}\mathbf{X}\mathbf{X}^T \\
=& \frac{1}{N_\text{samples}}\mathbf{USV}^T\mathbf{VSU}^T \\
=& \frac{1}{N_\text{samples}}\mathbf{US}^2\mathbf{U}^T \\
\end{align}
which is an eigenvalue decomposition of the covariance matrix (up to the scaling). Therefore the transformed basis vectors ${\mathbf U}$ in SVD and PCA are identical, and the eigenvalues and singular values are related as $\lambda_i = \sigma_i^2/N_\text{samples}$. Consistent with the previous definitions, the scores in PCA are given by $\mathbf{Z=U}^T\mathbf{X=U}^T\mathbf{USV}^T=\mathbf{SV}^T$. Therefore, in the language of SVD, dimensionality reduction with PCA is equivalent to truncating the decomposition after $R$ singular values. This means taking only the first $R$ columns of ${\mathbf U}$ and ${\mathbf V}$ as well as the upper $R\times R$ corner of $\mathbf S$ to get the following rank-$R$ approximation:
$$ \mathbf{\hat X} = \mathbf {U}_R \mathbf{S}_{R} \mathbf{V}_R^T$$

Further, the [Eckart-Young Theorem](https://rich-d-wilkinson.github.io/MATH3030/3-5-lowrank.html) states that the truncated SVD approximation above gives the rank-$R$ that is closest to the data matrix in the Frobenius norm:
$$\min_{\text{rank}(\mathbf{Y}) \leq R} \lVert  \mathbf{X - Y} \rVert_F$$
with $\lVert  \mathbf{X - \hat X} \rVert_F = \left(\sum_{i=R+1}^N \sigma_i^2\right)^\frac{1}{2}$. Equivalently, this means that taking the top $R$ principal components PCA results in the best low-rank approximation to the data with the following MSE:
$$E = \frac{1}{N \cdot N_\text{samples}}\sum_{i=R+1}^{N}\sigma_i^2 $$

Together with the maths demonstrated above, this shows that there is a deep connection between minimizing the error of a low rank (or low-dimensional) approximation and capturing covariability patterns in neural data. In the following section we will discuss this idea in extension to neural data tensors.

## From matrices to tensors
<a id='tensors'></a>
So far, we have focused on reducing the dimensionality of a matrix $\mathbf{X}\in \mathbb{R}^{N \times K}$. However, in neuroscience we often work with a data *tensor* $\mathcal{X}\in \mathbb{R}^{N \times T \times K}$, where $N$ is the number of neurons, $K$ the number of *trials* and $T$ the number of time points.

The first key element to notice, is that, unlike for matrices, we cannot naturally separate the variables at hand into *features* (neurons) and *samples* (time points). Instead there are two relevant notions of sampling (across time points and across trials) which we would like to keep separated to interpret the data [4]. In the following sections, we will look at three complementary ways to account for this, from which stem three dimensionality reduction methods : PCA on unfoldings [3], TCA [4] and sliceTCA [5].

### PCA on unfoldings

#### Neuron-unfolding

The first strategy, is to go back to the matrix case by *unfolding* the data tensor. For example, if we wish to keep neurons as features, we can unfold the tensor $\mathcal{X}\in \mathbb{R}^{N \times T \times K}$ into a matrix $\mathbf{X}\in \mathbb{R}^{N\times (TK)}$. If we now apply the low-rank approximation perspective on PCA to this matrix, the data can be approximated as:

$$
  \mathbf{X}\approx \mathbf{\hat X} = \mathbf{U}\mathbf{Z} \in \mathbb{R}^{N\times (TK)}
$$

where $\mathbf{U}\in\mathbb{R}^{N\times R}$ and $\mathbf{Z}\in \mathbb{R}^{R \times(TK)}$. As seen previously, this can be equivalently written as the sum of the following outer products:

$$
\mathbf{\hat X} = \sum_{r=1}^R \mathbf{u}_r\mathbf{z}^T_r
$$

where $\mathbf{u}_r \in \mathbb{R}^N$ are the columns of $\bf U$ and $\mathbf{z}_r\in\mathbb{R}^{TK}$ are the rows of $\bf V$.


Next, we can *fold* each $\mathbf{z}_r$ back into a matrix $\mathbf{W}_r\in\mathbb{R}^{T \times K}$. The outer product $\mathbf{u}_r\mathbf{z}^T_r$ is sometimes written in the form $\mathbf{u}_r\otimes \mathbf{z}_r$, which allows us to generalize the notation for the outer product of two vectors $(\mathbf{u}_r\otimes \mathbf{z}_r)_{ij}=\mathbf{u}_{r,i}\mathbf{z}_{r,j}$  to the outer product of a vector with a matrix $(\mathbf{u}_r\otimes \mathbf{W}_r)_{ijk}=\mathbf{u}_{r,i}\mathbf{W}_{r,jk}$. The figure below shows how a single *component* can be seen as either the outer product of a vector with a matrix or with a (long) vector.

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1oxa02j3jDSrY2izD9M_bpoHtWa6Ja6I1" width="500px;">
</p>

This way, we can rewrite our (SVD/PCA) approximation directly as a tensor:

$$
\mathcal{X} \approx \mathcal{\hat X} = \sum_{r=1}^R \mathbf{u}_r \otimes \mathbf{W}_r \in \mathbb{R}^{N \times T \times K}
$$

And, since the Frobenius norm is defined element-wise, the same objective as for the matrix case can be used to determine the decomposition:

$$
E(\mathbf{u}_1,..., \mathbf{u}_r, \mathbf{W}_1, ..., \mathbf{W}_r) = \lVert \mathcal{X} - \mathcal{\hat X} \rVert_F
$$

#### Trial-unfolding

We arbitrarily decided that the *neuron* leg would be the features (the $\mathbf{u}$'s). When working with matrices, this doesn't matter; in the end, the term $\mathbf{u}_r \mathbf{z}_r^T$ is symmetric, in the sense that it can be transposed (i.e. swap $\mathbf{u}_r$ and $\mathbf{z}_r$) to now consider the time points as features. When moving to tensors, this doesn't work anymore, as $\mathbf{z}_r$ now represents two variables (e.g. time and trial).

In the context of tensors, if we consider *time* as our features, we just have to unfold the tensor along another of its *legs*:

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1dHfvXK4ay3gZHx2_Om35MiRgWrOcNIDY" width="500px;">
</p>

And similarly for trials [3]:

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1AOPTNowx6LejxaGaMJGjqD58_KEBJcvE" width="500px;">
</p>

To keep track of the different vectors and matrices at hand, we can indicate the leg(s) of the tensor they describe as a supsercript. For trial-unfolding PCA, we would therefore write:

$$
\mathcal{X} \approx \mathcal{\hat X} = \sum_{r=1}^R \mathbf{u}_r^{trial} \otimes \mathbf{W}^{neuron, time}_r
$$

Note that, formally, $\mathbf{u}_r^{trial} \otimes \mathbf{W}^{neuron, time}_r \in \mathbb{R}^{K \times N \times T}$, but we have implicitely permuted the indices to match the data tensor in $\mathbb{R}^{N \times T \times K}$.

### (bonus) SliceTCA

We have just seen that applying PCA on unfoldings of a data tensor was equivalent to writing the approximation of the data tensor as a sum of outer product of vectors (the feature) and matrices (the folded scores vectors). We can generalize this concept by allowing different types of unfoldings within the same sum. Preserving the same notation as before,

$$
\mathcal{X} \approx \mathcal{\hat X} = \sum_{r=1}^{R_{neuron}} \mathbf{u}_r^{neuron} \otimes \mathbf{W}^{time, trial}_r  + \sum_{r=1}^{R_{time}} \mathbf{u}_r^{time} \otimes \mathbf{W}^{neuron, trial}_r  + \sum_{r=1}^{R_{trial}} \mathbf{u}_r^{trial} \otimes \mathbf{W}^{neuron, time}_r
$$

This decomposition is called sliceTCA [5].

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1H5RkoRhITPLbAGHiLMKaK_WsQqk-WzfM" width="600px;">
</p>

Note that by setting (e.g.) $R_{time}=R_{trial} = 0$, we fall back onto (e.g.) the neuron-unfolding PCA decomposition. So that sliceTCA is a generalization (or in a set-theoretic sense, a superclass) of PCA on unfoldings.

### (bonus) TCA

Another strategy for going from PCA on unfoldings to a tensor decomposition is to constrain the slices to be of rank $1$. Consider again a PCA on (any) unfolding decomposition:

$$
\mathcal{X} \approx \mathcal{\hat X} = \sum_{r=1}^R \mathbf{u}_r \otimes \mathbf{W}_r \in \mathbb{R}^{N \times T \times K}
$$

If we now constrain the matrix to be of rank $1$, that is $\mathbf{W}_r = \mathbf{v}_r \mathbf{w}_r^T = \mathbf{v}_r \otimes \mathbf{w}_r$:

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1STP28EUzPHYZNEkh0uC0PiUPmf4W1fx5" width="500px;">
</p>

And we thus obtain the decomposition:

$$
\mathcal{X} \approx \mathcal{\hat X} = \sum_{r=1}^R \mathbf{u}_r \otimes \mathbf{v}_r \otimes \mathbf{w}_r \in \mathbb{R}^{N \times T \times K}
$$

This decomposition is known within neuroscience as tensor component analysis (TCA, [4]) but in other fields (such as physics) is known by other names: CANDECOMP, PARAFAC, or simply the CP decomposition. Therefore, PCA is a generalization (or superclass) of TCA.

### Relationship between tensor decomposition methods

Overall, the relationsip between the three tensor decomposition methods presented here can be summarized by the following Venn diagram [5]:

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=1mNdFEP7gnDK72Q-jDyT1jkKs1sG4q698" width="300px;">
</p>


# **Part II.** Vizualizing and denoising data with PCA on a toy data


To familiarize yourselves with PCA/SVD, you will first write your own implementation of PCA (without using `sklearn`). Second, you will try to *guess* the rank of some simulated data using the bi-cross-validation method.

**WARNING:** In the maths above we used the convention that $\mathbf{X} \in \mathbb{R}^{N\times K} $ (i.e., the columns are the samples), as this is commonly used in neuroscience and in the PCA literature. However, other communities (notably in machine learning) use the rows as samples instead: $\mathbf{X} \in \mathbb{R}^{K\times N}$. Be mindful when you write your code that the packages you use may be using the latter notation (for example, in that case, the $\bf V$ in the SVD will represent the basis vectors, rather than the $\bf U$.).

First, execute the cell below to get started and generate your data matrix $M$ of 10 neurons and 101 time points.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(3)

def exp_quad(x, l=0.1, s=1): 
    return np.exp(-x**2/(2*l**2))*s**2

n = 10
k = np.random.randint(1, n//2+1)
time_steps = 101
ts = np.linspace(0, 1, time_steps)
noise = 0.01
kernel = exp_quad

latent_gaussian = np.random.randn(time_steps, k)
covariance_matrix = kernel(np.subtract.outer(ts, ts))
L = np.linalg.cholesky(covariance_matrix+10**-6*np.eye(time_steps))

neuron_basis = np.random.randn(n, k)
time_basis = L @ latent_gaussian

M = time_basis @ neuron_basis.T + np.random.randn(n) + np.random.randn(time_steps, n)/3

print('Data matrix M (sample_size, neuron):', M.shape)

**Step 1.** The first step of neural dimensionality reduction is *always* to look at the raw data first. Plot the single neuron activity generated by the previous code cell over time.

In [ ]:
# Fill in your code below
fig = plt.figure(figsize=(6,3), constrained_layout=True)
ax = fig.add_subplot()

ax.plot( # your code here)
ax.set_xlabel('Time')
ax.set_ylabel('Neural activity')
plt.show()

**Step 2.** Center the data matrix (subtract from each neuron its mean activity). Make a scatter plot of two arbitrarily chosen neurons, using each as one of the two dimensions. What do you notice?

In [ ]:
fig = plt.figure(figsize=(4,4), constrained_layout=True)
ax = fig.add_subplot()

# Fill in your code below
###########################

# Center each neuron activity in M
M_centered = # your code here

# Plot the centered activity for neurons i and j of your choise
ax.scatter()

###########################

ax.set_aspect('equal')
ax.spines[['left', 'bottom']].set_position(("data", 0))
ax.spines[['top', 'right']].set_visible(False)
ax.set_xlabel('Neuron '+str(i), loc='right')
ax.set_ylabel('Neuron '+str(j), loc='top')
plt.show()

**Step 3.** Perform PCA using ``` np.linalg.svd ```.  Be careful with rows and columns, and verify whether the basis vectors should be $\bf U$ or $\bf V$. Make the following two plots:
* a scatter plot of projections of the data onto the first two PCs using the $S$, $V$, and $U$.
* a plot showing how the projected variance decays as a function of the number of components.

In [ ]:
# Fill in your code below
###############################

# Generate U, S, V using SVD
U, S, V = # your code here

# Generate the projected data
projection = # your code here

# Compute the variance along each dimension (careful with matrix axes!)
projected_variance = # your code here

###############################

fig = plt.figure(figsize=(8,3), constrained_layout=True)
ax = fig.add_subplot(1,2,1)

ax.scatter(projection[:,0], projection[:,1], alpha=0.5)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

ax = fig.add_subplot(1,2,2)
ax.plot(projected_variance,'o-')
ax.set_xlabel('Number of components')
ax.set_ylabel('Projected variance')
plt.show()

**Step 4.** Now make the same plot as above using the eigenvectors of the covariance matrix.
* Calculate the sample covariance matrix.
* Use ``` np.linalg.eig ```  to get the eigenvectors and eigenvalues of the covariance matrix. Be careful with the rows and columns.
* Make a scatter plot of the data matrix on the first two eigenvectors of the covariance matrix, just as you did in Step 3. Are the two plots identical? Should they be?
* Finally plot the projected variance using the eigenvalues. Are the two plots identical?

In [ ]:
# Fill in your code below
###############################

# Generate the covariance matrix
Sigma = # your code here

# Compute the eigenvalues using np.linalg.eig()
evals, evecs = # your code here

# Generate the projected data
projection = # your code here

# Compute the variance using eigenvalues
projected_variance = # your code here

###############################

# Let's plot what we got
fig = plt.figure(figsize=(8,3), constrained_layout=True)
ax = fig.add_subplot(1,2,1)

ax.scatter(projection[:,0], projection[:,1], alpha=0.5)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

ax = fig.add_subplot(1,2,2)
projected_variance = np.sort(projected_variance)[::-1]
ax.plot(projected_variance,'o-')
ax.set_xlabel('Number of components')
ax.set_ylabel('Projected variance')
plt.show()

**Step 5.** PCA is often used as a preprocessing step for other dimensionality reduction methods due to its denoising properties. Let's try to recompose the neural firing rates using the first two principal components. Does it look denoised to you? Try to change the number of components to see the difference.<br>


In [ ]:
n_components = 2

# Fill in your code below
##############################

# let's reconstruct the data by using only few components (always remember to put back the mean)
M_reconstructed = # your code here

##############################

fig = plt.figure(figsize=(10,3), constrained_layout=True)
ax = fig.add_subplot(1,2,1)

ax.set_title('Initial data')
ax.plot(ts, M)
ax.set_xlabel('Time')
ax.set_ylabel('Neural activity')

ax = fig.add_subplot(1,2,2)

ax.set_title('Reconstructed data')
ax.plot(ts, M_reconstructed)
ax.set_xlabel('Time')
ax.set_ylabel('Neural activity')
plt.show()

**Step 6.** A frequently used method to calculate the dimensionality is to plot the cumulative fraction of variance explained in the data and see when the curve levels off. Since the projected variance is given by the eigenvalues, this can be evaluated by a cumulative sum of eigenvalues normalized by the sum of all eigenvalues. Plot this as a function of the number of components. What do you think the dimensionality is?

In [ ]:
# Fill in your code below
##############################

# Compute the comulative (remember to normalize)
cumulative = # your code here

##############################

fig = plt.figure(figsize=(4,3), constrained_layout=True)
ax = fig.add_subplot()
ax.plot(cumulative, 'o-')
ax.set_xlabel('Number of components')
ax.set_ylabel('Variance explained')
plt.show()

# **Part III.**  Cross-validating dataset dimensionality


A key step in dimensionality reduction methods is to determine the dimensionality of the data, that is to decide on the number of components to use. There exist two methods for doing it: (1) bi-cross-validation [6], which can be used for PCA, NMF, and similar matrix factorizations, and (2) masking entries of the data matrix or tensor during optimization, which is more general. Here, we will focus on ___bi-cross-validation___.

You can read more about cross-validation methods for PCA on this nice [blog post](https://alexhwilliams.info/itsneuronalblog/2018/02/26/crossval/).

## The method

### Bi-cross-validation for PCA

The SVD formulation of PCA allows for a very nice way to cross-validate the number of components [6]. The basic idea is that we will split the data into *two* test/training sets: one in neurons, and one in time. That is, we split the data matrix into block structure:
$\mathbf{X} = \begin{bmatrix} \mathbf{X}^{tr}_{tr} & \mathbf{X}_{tr}^{te}\\ \mathbf{X}_{te}^{tr} & \mathbf{X}^{te}_{te} \end{bmatrix}$.
The subscripts corresponds to neuron indices, and the superscripts to sample indices.

For each $R$, we use the truncated SVD on the data from *all* neurons during the *training* samples to identify the $R$ basis vectors:
$$\mathbf{X}^{tr} = \mathbf{U}_R\mathbf{Z}_{R}^{tr}$$
or, in block form:
$$\begin{bmatrix} \mathbf{X}^{tr}_{tr} \\ \mathbf{X}_{te}^{tr} \end{bmatrix} = \begin{bmatrix} \mathbf{U}_{R,tr} \\ \mathbf{U}_{R,te} \end{bmatrix} \mathbf{Z}_{R}^{tr}$$

We now want to cross-validate how well $\mathbf{U}_R$ describes the data in the test samples. This is in general ill-defined since we are moving from a low-dimensional subspace to a high-dimensional space, but the key idea is to use the *training* neurons in the *test* samples, in addition to the low-dimensional basis we just identified from the *training* samples, to predict the *test* neuron's activity in the *test* samples. Starting from the equivalent block formula for the test samples, we have:
$$\begin{bmatrix} \mathbf{X}^{te}_{tr} \\ \mathbf{X}_{te}^{te} \end{bmatrix} = \begin{bmatrix} \mathbf{U}_{R,tr} \\ \mathbf{U}_{R,te} \end{bmatrix} \mathbf{Z}_{R}^{te}$$

We first take advantage of the top block of the equation, using the basis we just identified, to obtain an estimate for the scores during the test samples $\mathbf{\hat Z}_{R}^{te}$, i.e. we solve the following linear regression problem:

$$\mathbf{X}^{te}_{tr} = \mathbf{U}_{R,tr}\mathbf{Z}_{R}^{te}$$
Now we plus this into the bottom block of the equation to obtain our estimate for the test neurons during test samples:
$$\mathbf{\hat X}^{te}_{te} = \mathbf{U}_{R,te}\mathbf{\hat Z}_{R}^{te}$$
We can now calculate the fraction of explained variance in the estimate as we [normally would](https://en.wikipedia.org/wiki/Fraction_of_variance_unexplained#:~:text=In%20statistics%2C%20the%20fraction%20of,by%20the%20explanatory%20variables%20X.):
$$ \text{frac var exp} = 1 - \frac{\text{MSE}}{\text{var}}$$
We repeat for different $R$ and as usual look for the value that maximizes the cross-validated explained variance.

There are a couple practicalities to note from this method that stem from the linear regression problem. First, note that the number of neurons in the training set has to be greater than $R$, or else the linear regression problem is underdetermined. This puts a limit to what dimensionalities we can test depending on how many neurons we can afford to use for training. Second, since we are using a subset of neurons to predict activity in other neurons, this method does not work well if the test neurons are independent of the training neurons. In other words it assumes some amount of shared variability between the neurons.

However, this formulation of bi-cross-validation is very particular to matrix decompositions and does not extend easily to tensors or to nonlinear methods.

### Cross-validation for tensors

To cross-validate other methods, we can simply hold out or mask entries of the data matrix/tensor while fitting the parameters of the model (e.g. the $\mathbf{u}$'s) on the remaining entries [4]. However, we must be careful to take into account temporal correlations since the tensor entries are not independently generated. That is, if we hold out single entries randomly throughout the matrix, the training and testing dataset will be strongly correlated. In fact, assuming the data is continuous in time (as would be the case for imaging data, or with firing rate estimates), with a sufficiently high sampling rate (i.e. sufficiently small time bins), the entry $\mathcal{X}_{i,j,k}$ of the data tensor (supposed to be in the test set) can be arbitrarily well approximated by averaging the $\mathcal{X}_{i,j-1,k}$ and $\mathcal{X}_{i,j+1,k}$ (which may both be in the train set). Instead, we advocate for holding out blocks and trimming the ends for the most fair separation between test and training data [5]. What timescale should we use? Some good bets are the timescale of the calcium indicator (imaging data) the smoothing kernel (for ephys). Or we can simply use the autocorrelation timescale for any arbitrary time series.

<p align="center" width="100%">
<img src="https://drive.google.com/uc?id=13dBa2-ccAWAeXf9oasoNJTO3bNvfPJOW" width="300px;">
</p>

## Application of PCA bi-cross validation on a toy dataset

Now we will move on to a more realistic  model, where the true rank of the data is unclear due to noise. We will use ___PCA bi-cross-validation___ to try to guess the cross-validated dimensionality. For this we will generate higher-dimensional data. Execute the following code to generate your new data matrix $M$ of 100 neurons and 500 time points to get started.

*(Note that for time series data we would need to be careful about temporal correlations between adjacent entries. In this particular tutorial we draw the samples from a multivariate Gaussian with no notion of time so we don't have to worry about it!)*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

np.random.seed(1)

n = 100
k = np.random.randint(1, n//3+1) # Mystery rank
sample_size = 500
alpha = 0.0 # Noise term

u = np.random.randn(n, k)/np.sqrt(n*k)
v = np.random.randn(n, n)/np.sqrt(n*k)
C = u @ u.T + alpha*(v @ v.T) # Positive semi-definite symmetric matrix <= covariance matrix

M = np.random.multivariate_normal(np.random.randn(n)/np.sqrt(n), C, sample_size) + np.random.randn(sample_size, n)/10

print('Shapes:')
print('  Data matrix (sample_size, neuron):', M.shape)
print('  Covariance matrix (neuron, neuron):', C.shape)

**Step 0.** Before starting, it is useful to define some helper functions, computing the mean squared error (MSE) and the explained variance, since we will use them a lot.

In [ ]:
def compute_mse(prediction, target):
    """
    Function computing the mean squared error between predicted and target data
    Inputs: 
        prediction (numpy array): predicted values
        target (numpy array): target values
    returns:
        mse (float): mean squared error
    """
    # Fill in your code below
    ##############################
    return # your code
    ##############################
 
def compute_var_exp(prediction, target):
    """
    Function computing the variance explained between predicted and target data.
    It is usually defined as 1 - MSE/mean(target^2)
    Inputs: 
        prediction (numpy array): predicted values
        target (numpy array): target values
    returns:
        var_exp (float): variance explained
    """
    # Fill in your code below
    ##############################
    return # your code
    ##############################

**Step 1.** Plot the cumulative fraction of variance explained in $M$ as a function of the number of principal components. You can use either the eigendecomposition or the SVD, as you prefer. Can you guess what the dimensionality is?

In [ ]:
# Fill in your code below
##############################

# Center the data
M_centered = # your code

# Compute the explained variance
# your code here
#

# (if you use eigendecomposition, don't forget to sort the eigenvalues)
# your code here

# compute the cumulative fraction of variance explained (normalized)
cumulative = # your code here

##############################

fig = plt.figure(figsize=(6,3), constrained_layout=True)
ax = fig.add_subplot()
ax.plot(cumulative,'o-')
plt.show()


**Step 2.** Now let's start the bi-cross-validation. Do the following steps.
* First (we already did it for you), split the data into training and test sets in both time and trials. We have written an example code using ```train_test_split``` from ```sklearn``` but feel free to modify / implement it your own way. Use 80% for training both in neurons and in time. Make sure it's clear to keep track of which indices are neurons and which are time.
* Second, choose a specific rank $r$ to first test your code. (But remember that it can't be larger than the number of neurons used for training)
* Third, apply PCA on the data for training samples (all neurons) using SVD.
* Next, use linear regression to solve for the scores in the test samples. You might want to use ```np.linalg.lstsq```.
* Finally, make a scatter plot of the actual data for the training neurons, test samples against the values you predicted from PCA and linear regression (also for training neurons, test samples - so it's not yet cross validated). Calculate and print the fraction of variance explained. How accurate is it?

In [ ]:
# Creating the indeces for selecting train/test sets over samples/neurons
neur_indices_tr,neur_indices_te = train_test_split(np.arange(M.shape[1]), train_size = .8, shuffle=True)
time_indices_tr,time_indices_te = train_test_split(np.arange(M.shape[0]), train_size = .8, shuffle=True)

# Selecting the training and test data
M_tr_all = M_centered[time_indices_tr,:] # Note that data must be centered before!
M_te_all = M_centered[time_indices_te,:]
M_te_tr = M_te_all[:,neur_indices_tr]
M_te_te = M_te_all[:,neur_indices_te]


# Fill in your code below
############################

# Choose the rank r
# your code here

# Apply SVD on the training data
# your code here

# Now, we solve the scores with linear regression. Let's use 'u' (small) for the train basis, 
# 'z' for the scores and 'x' forthe data
u = # your code here
x = # your code here

# Use linear regression ( u z = x) to solve the scores on test data
z = np.linalg.lstsq(u,x,rcond=None)[0]

# Compute the prediction
M_te_tr_predict = # your code here

############################

# Compute variance explained
var_exp_tr = compute_var_exp(M_te_tr_predict,M_te_tr)

fig = plt.figure(figsize=(4,3), constrained_layout=True)
ax = fig.add_subplot()
ax.scatter(M_te_tr_predict.flatten(), M_te_tr.flatten(),alpha=0.5)
ax.axline((0,0), (0.7,0.7), color='black', linestyle='--')
print(var_exp_tr)

**Step 3.** Now we will try to do the cross validation to the test neurons, test samples data.
* Predict the activity of the test neurons in the test dataset. Make the same plot as in the last cell, and calculate the cross-validated fraction of variance explained. How does it compare to the previous step?
* Repeat this and the last cell for a few different choices of rank. What happens when the rank is very small ? Very large ?

In [ ]:
# Fill in your code below
##########################

# Compute the prediction on test neurons 
M_te_te_predict = # your code here

# Compute variance explained
var_exp_te = # your code here

##########################

fig = plt.figure(figsize=(4,4), constrained_layout=True)
ax = fig.add_subplot()
ax.scatter(M_te_te_predict.flatten(), M_te_te.flatten(),alpha=0.5)
ax.axline((-0.5,-0.5), (0.5,0.5), color='black', linestyle='--')
plt.show()

print("Variance explained (test): ",var_exp_te)

**Step 4.** Here we will do the bi-cross-validation systematically to try to guess the rank.
* Copy/paste the code above into a loop to calculate the fraction of variance explained (both in training and test neurons) for different numbers of components. Test all possible dimensionalities.
* Plot the variance explained in both the training and test neurons over the number of components. What's the dimensionality?

In [ ]:
# enter here your guess for the rank
your_guess = 3


r_max = M_te_tr.shape[1]
dims = np.arange(1,r_max+1)
var_exp_tr = np.empty(dims.shape)
var_exp_te = np.empty(dims.shape)

for r in dims:
    # Reuse the last two cells of code for computing train/test explained variance
    # Fill in your code below
    ###########################

    # Apply SVD on the training data
    U, S, V = np.linalg.svd(M_tr_all, full_matrices=False)

    # Now, we solve the scores with linear regression. Let's use 'u' for the train basis, 
    # 'z' for the scores and 'x' forthe data
    u = # your code here
    x = # your code here
    # Use linear regression ( u z = x) to solve the scores on test data
    z = np.linalg.lstsq(u,x,rcond=None)[0] # scores

    # Compute the prediction for training neurons
    M_te_tr_predict = # your code here

    # Compute explained variance
    var_exp_tr[r-1] = compute_var_exp(M_te_tr_predict,M_te_tr)

    # Compute the prediction on test neurons
    M_te_te_predict = # your code here

    # Compute variance explained
    var_exp_te[r-1] = compute_var_exp(M_te_te_predict,M_te_te)
    ###########################

fig = plt.figure(figsize=(4,3), constrained_layout=True)
ax = fig.add_subplot()
ax.plot(dims,var_exp_tr,'-o', label = "train data")
ax.plot(dims,var_exp_te,'-o', label = "test data")
ax.legend()
ax.set_xlabel("Rank")
ax.set_ylabel("Variance explained")
ax.set_ylim([0,1.05])
plt.show()

k_guess = dims[np.argmax(var_exp_te)]

print("Computed rank: ", k_guess)
print("Real rank (the mistery rank k):", k)
print('Your guess was '+('Correct !' if your_guess == k_guess else 'Incorrect !'))

# **Part IV.** Application to neural data analysis


The data consists of microelectrode array recordings from M1 and PMd during a delayed center-out reach task. It is one of the datasets from the [Neural Latent Benchmark](https://neurallatents.github.io/) project, whose aim is to provide standardized freely accessible neural datasets for the sake of developing data analysis and modeling methods.



---



The task is a delayed center-out reach task with barriers, resulting in a variety of straight and curved trajectories. Neural activity was recorded from the dorsal premotor and primary motor cortices, and cursor, monkey gaze position, and monkey hand position and velocity are also provided. You can find more information on the dataset in the articles [7-8].


<p align="center" width="100%">
<img src="https://neurallatents.github.io/assets/maze_fig1.png" width="550px;">
</p>

---
It has been found that M1/PMd activity play a key role in motor control during this type of tasks.
___Today, we will explore this hypothesis by analyzing how the activity of the population of neurons is able to decode behavioral information.___

## Data imports

The neural data is presented in the Neurodata Without Border (NWB) format, which is a commonly used format for storing and sharing neural data. Here, all the code was written for you, you may just run the following cells, they will download the part of the dataset from this experiment which is relevant to this tutorial: one recording session. <br>
___Note that this can take a while (<5min) to run. So you should avoid restarting your runtime environment or rerunning those cells.___

### Run this (offline users)

__We install some packages using pip. You will need to adapt the first cell code if you use conda!__

#### Import/install necessary libraries

In [ ]:
# You should not evaluate this cell multiple times

from IPython.display import clear_output

# Download the data
!pip install pandas
!pip install --upgrade numpy
!pip install git+https://github.com/neurallatents/nlb_tools.git
!pip install dandi
!pip install pydantic[email]

clear_output() #clear cell output

In [ ]:
from nlb_tools.nwb_interface import NWBDataset
from IPython.display import clear_output
import numpy as np
import pandas as pd
import pickle

np.random.seed(10)

#### Download/load the raw dataset

The continuous data provided with the MC_Maze datasets includes:
* `cursor_pos` - x and y position of the cursor controlled by the monkey
* `eye_pos` - x and y position of the monkey's point of gaze on the screen, in mm
* `hand_pos` - x and y position of the monkey's hand, in mm
* `hand_vel` - x and y velocities of the monkey's hand, in mm/s, computed offline using `np.gradient`
* `spikes` - spike times binned at 1 ms

In [ ]:
#The part of the dataset of interest

!dandi download https://gui.dandiarchive.org/#/dandiset/000128
dataset = NWBDataset("000128/sub-Jenkins/", "*train", split_heldout=False)
clear_output()

#### Dataset visualization (table)

In [ ]:
dataset.data

In [ ]:
dataset.trial_info

#### Dataset preparation

In [ ]:
#The part of the dataset of interest
bin_size = 5 # ms
time_window = (-1000, 800)
timesteps = int((time_window[1]-time_window[0])//bin_size)
n_neurons = len(dataset.data.spikes.columns)

# resample the dataset
dataset.resample(bin_size)
# align trials to movement onset
alldata = dataset.make_trial_data(align_field='move_onset_time', align_range=time_window)

trials = alldata.trial_id.unique()
clear_output()

In [ ]:
trials = alldata.trial_id.unique()

nns = np.zeros([len(trials),n_neurons, timesteps]) # here we will save the neural activity
hhs = np.zeros([len(trials), 4, timesteps]) # here we will save the hand position
info_df = [] # this will contain the trial info

names = ['trial', 'ttype', 'tversion', 'target_x', 'target_y', 'success', 'num_targets', 'active']

for t, df in alldata.groupby('trial_id'):
    nns[t] = df.spikes.values.T
    hhs[t] = np.concatenate([df.hand_pos.values.T, df.hand_vel.values.T],axis=0)

    info_df.append([
        dataset.trial_info.trial_id[t],
        dataset.trial_info.trial_type[t],
        dataset.trial_info.trial_version[t],
        dataset.trial_info.target_pos[t][0][0], 
        dataset.trial_info.target_pos[t][0][1],
        dataset.trial_info.success[t],
        dataset.trial_info.num_targets[t],
        dataset.trial_info.active_target[t]
	])

info_df = pd.DataFrame(info_df, columns=names)

# here we extract a part of the dataset for faster computations
## Note that we selected sparser targets, for visualization purposes
cond = [26, 29, 1, 13, 32, 28, 33, 23, 10, 6 ,20, 38]

info_df = info_df.loc[info_df.num_targets==1]
info_df = info_df.loc[np.isin(info_df.ttype, cond)]

info_straight = info_df.loc[info_df.tversion==0].reset_index(drop=True)
info_curved = info_df.loc[info_df.tversion==1].reset_index(drop=True)

hand_straight = hhs[info_straight.trial.values]
hand_curved = hhs[info_curved.trial.values]

neurons_straight = nns[info_straight.trial.values]
neurons_curved = nns[info_curved.trial.values]

info_straight.to_pickle('info_straight.p')
info_curved.to_pickle('info_curved.p')

np.save('hand_straight.npy', hand_straight)
np.save('hand_curved.npy', hand_curved)

np.save('neurons_straight.npy', neurons_straight)
np.save('neurons_curved.npy', neurons_curved)

del alldata, dataset, nns, hhs, info_df, info_straight, info_curved, hand_straight, hand_curved, neurons_straight, neurons_curved

### Run this (colab users)

In [ ]:
from IPython.display import clear_output

!pip install pandas
!pip install git+https://github.com/neurallatents/nlb_tools.git
!pip install gdown

# download the preprocessed data
!gdown https://drive.google.com/uc?id=1fhmEvzfelRuYn0ZWpoxTrk8eFLIqh3Jh -O adv-dim-red.zip
!unzip adv-dim-red.zip
!rm adv-dim-red.zip
!mv adv-dim-red-data/* .
!rm -r adv-dim-red-data

clear_output()

### Loading the dataset

Here, we load the neural data, behavioral data and extra trial data information. <br><br>
__We will mainly focus our analysis on the straight part of the dataset.__ <br><br>

If you are curious (and if you have some spare time once finished), you could also try to have a look at the other part of the dataset, where the monkeys need to reach the target while avoiding some obstacles. This force the trajectories to be longer, more curved and complex. You can find these data in the files `neurons_curved.py`, `hand_curved.py` and `info_curved.p`.

But let's focus on the simple case. Here, we load and print some info on the dataset. Just run the cell, no need to write anything here.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm

import scipy.ndimage as spnd

np.random.seed(10)


# loading neural and behavioral datasets
neural_data = np.load('neurons_straight.npy')
behavioral_data = np.load('hand_straight.npy')

# task variables
with open('info_straight.p', 'rb') as f:
    trial_data = pickle.load(f).reset_index()

# defining some useful constants
time_window = (-1000, 800) # ms
bin_size = 5 # ms
onset_time = -time_window[0]//bin_size
n_trials, n_neurons, timesteps = neural_data.shape
time_axis = np.arange(time_window[0], time_window[1], bin_size)


print('\033[1;4m# Data format\033[0m\n')
print('Time window:', time_window, 'ms\n')
print('Bin size:', bin_size, 'ms\n')
print('Movement onset: 0 ms\n')
print('Neural data shape (trials, neurons, time bins):\n', neural_data.shape)
print('\nBehavioral data shape (trials, x/y position + x/y velocity, time bins):\n', behavioral_data.shape)
print('\ntrial_data (Dataframe) columns\n',list(trial_data.columns))


### Some preprocessing

Before starting our analysis, we will need some preprocessing on the data. <br>
First, since we have several targets, we need a simple variable for labeling each trial. We will use the x/y target positions to extract the target angle, our label. For helping us plotting, we will assign a specific color for each target angle. This part is already coded.

Then, we need to convert the PSTH dataset into firing rates estimates. A simple way to do it is to ___apply a gaussian filter on it, applied on the time axis for each specific neuron___. 
- Since our dataset might not be binned at 1ms (info in the cell above), first compute the gaussian std in bins that you want to use for the smoothing. We will use a std of 50 ms.
- Apply the smoothing on the neural data, saving it in a variable called `rate_neural_data`. If you don't know how to do the smoothing, you can try having a look at `scipy.ndimage.spnd.gaussian_filter1d()`

In [ ]:
# calculate target vangle and get index for sorting trials by target angle
trial_data['angle'] = np.arctan2(trial_data.target_y.values, trial_data.target_x.values)

# Here, we define a color map for the different targets. 
# You can use 't_map[angle]' for getting the color
targets = np.sort(trial_data['angle'].unique())
colors = matplotlib.cm.rainbow(np.linspace(0,1,len(targets)))
t_map = {angle: colors[i] for i, angle in enumerate(targets)}

# Fill in your code below
##############################
# Gaussian filter time window
gaussian_filter_std = 50 # ms

# convert the std in bins size
gaussian_filter_std_bins = # your code here

# smooth along time axis, using a Gaussian filter (careful with the axis)
rate_neural_data = # your code here

## Simple statistics

### Analysis of behavior

__Step 1 (Hand Trajectories).__ First, let's plot the hand trajectories. Try to make two subplots where trajectories are color coded by the target angle (present in the trail_data).
- In the first plot, plot all the color coded trajectories
- in the second plot, just plot the mean trajectory for each target angle

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6), dpi=60,constrained_layout=True)

# let's extract the angles from trial_data
angles = trial_data['angle'].values

# Here, we define a color map for the different targets. You can use 't_map[angle]' for getting the color
targets = np.sort(trial_data['angle'].unique())
colors = matplotlib.cm.rainbow(np.linspace(0,1,len(targets)))
t_map = {angle: colors[i] for i, angle in enumerate(targets)}

for trial_id,trial_traj in enumerate(behavioral_data):
    # Fill in your code below
    ###############################
    # remember that trials and angles are in the same order
    angle = # your code

    ax[0].plot() # your code here
    ###############################

ax[0].set_xlabel('x (mm)')
ax[0].set_ylabel('y (mm)')
ax[0].set_title('Hand movement, different trials')

for angle in targets:
    # Fill in your code below
    ###############################
    # create the index vector, for selecting the trajectories
    indeces = # your code here

    # extract the trajectories and compute the mean over trials
    mean_trajectory = # your code here

    ax[1].plot() #your code here
    ###############################

ax[1].set_xlabel('x (mm)')
ax[1].set_ylabel('y (mm)')
ax[1].set_title('Hand movement, average trajectory')


plt.show()

As you can see, the animal perform stereotyped, slightly curved trajectories.

__Step 2 (Hand Velocity).__ Let's now plot the hand velocity. You can reuse the same code, just selecting the indeces in the behavioral_data matrix corresponding to velocity.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6), dpi=60,constrained_layout=True)

# let's extract the angles from trial_data
angles = trial_data['angle'].values

# Here, we define a color map for the different targets.
# You can use 't_map[angle]' for getting the color
targets = np.sort(trial_data['angle'].unique())
colors = matplotlib.cm.rainbow(np.linspace(0,1,len(targets)))
t_map = {angle: colors[i] for i, angle in enumerate(targets)}

for trial_id,trial_traj in enumerate(behavioral_data):
    # Fill in your code below
    ###############################
    # remember that trials and angles are in the same order
    angle = # your code

    ax[0].plot() # your code here
    ###############################

ax[0].set_xlabel('x (mm/s)')
ax[0].set_ylabel('y (mm/s)')
ax[0].set_title('Hand velocity, different trials')

for angle in targets:
    # Fill in your code below
    ###############################
    # create the index vector, for selecting the trajectories
    indeces = # your code here

    # extract the trajectories and compute the mean over trials
    mean_trajectory = # your code here
    ###############################
    ax[1].plot()

ax[1].set_xlabel('x (mm/s)')
ax[1].set_ylabel('y (mm/s)')
ax[1].set_title('Hand velocity, average')


plt.show()

Much more messy right? Before moving on, just try to spot the main differences between trajectories and velocities! What do you see?

__Step 3.__ Here, we plot the single x or y velocities, for gaining some information on movement timing.

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(5,7), dpi=100, constrained_layout=True)

chosen_angles = targets[[2,5,10]]

for a in chosen_angles:

    # Fill in your code below
    ############################

    # create the index vector for selected angle and extract the trajectories
    indeces = # your code here
    velocities = # your code here

    ### Plotting hand mean velocity

    # compute the velocity module, mean/std over trials
    velocity_module = # your code here
    mean_module = # your code here
    std_module = # your code here

    ############################

    # let's plot mean and std
    ax[0].plot(time_axis, mean_module, 
               c=t_map[a], linewidth=1, label=f'${a/np.pi*180:.2f}^\circ$')
    ax[0].fill_between(time_axis, 
                       mean_module-std_module, 
                       mean_module+std_module, 
                       color=t_map[a], alpha=0.2)

    ### Plotting hand x-velocity

    # Fill in your code below
    ############################

    # compute the x_velocity module, mean/std over trials
    x_velocity = # your code here
    x_velocity_mean = # your code here
    x_velocity_std = # your code here

    ############################

    # 
    ax[1].plot(time_axis, x_velocity_mean, 
               c=t_map[a], linewidth=1, label=f'${a/np.pi*180:.2f}^\circ$')
    ax[1].fill_between(time_axis, 
                       x_velocity_mean-x_velocity_std, 
                       x_velocity_mean+x_velocity_std, 
                       color=t_map[a], alpha=0.2)

    ### Plotting hand y-velocity

    # Fill in your code below
    ############################

    # compute the x_velocity module, mean/std over trials
    y_velocity = # your code here
    y_velocity_mean = # your code here
    y_velocity_std = # your code here

    ############################

    ax[2].plot(time_axis, y_velocity_mean, 
               c=t_map[a], linewidth=1, label=f'${a/np.pi*180:.2f}^\circ$')
    ax[2].fill_between(time_axis, 
                       y_velocity_mean-y_velocity_std, 
                       y_velocity_mean+y_velocity_std, 
                       color=t_map[a], alpha=0.2)

# Some settings
ax[0].set_title('Hand velocity module')
ax[0].set_ylabel('|Velocity| (mm/s)')
ax[1].set_title('Hand x-velocity')
ax[1].set_ylabel('Velocity (mm/s)')
ax[2].set_title('Hand y-velocity')
ax[2].set_ylabel('Velocity (mm/s)')
for i in range(3):
    ax[i].vlines(0, -700,1400, color='k', linestyle='--', linewidth=1, label = 'mov. onset',zorder=3)
    ax[i].set_ylim(-700,1400)
    ax[i].legend(loc='upper left')
    ax[i].set_xlabel('Time (ms)')
plt.show()

What do you observe? Is there some parts of the data data might not be useful to include for our analysis?

### Analysis of the neural data

#### Neuron-wise average

Plot a histogram of the average number of spikes per trial for all neurons.

In [ ]:
# Fill in your code below

*  What do you observe? (You can use more (~50) bins for improving visualization)

#### Time-wise average

We plot now the average (over neurons and trials) number of spikes as a function of time, and the same with its standard deviation to compare them.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,4),constrained_layout=True)

# Fill in your code below
##############################

# Compute the mean and standard deviation of the activity
mean_activity = # your code here
std_activity = # your code here

# plot the mean here
ax[0].plot() # your code here

# same as above
ax[1].plot() # your code here

# plot the std
ax[1].fill_between() # your code here

##############################


# other settings
ax[0].axvline(0, color='red', label='mov. onset', linestyle='--')
ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('Average number of spikes')
ax[0].legend()
ax[1].axvline(0, color='red', label='mov. onset', linestyle='--')
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Average number of spikes')
ax[1].legend()

plt.show()

*  What do you observe? Could we guess something from this plot?

### Trial to trial variability

As we just saw in the last plot, there is a lot of variability in the data, at basically any time. A good way to improve our understanding on it is to look at the neural activity more closely, plotting the PSTH for some neurons, as well as the per-target average firing rate over time.


___Sort the spikes according to the target angle, in order to put together activity for the same target. This will help reading the data.___

(You might want to check out other neurons once you have the code correctly running)


In [ ]:
neurons = [0,30,76]

# Fill in your code here
###############################

# first we want to sort the spikes according to the target angle
sorted_trial_data = # your code here

# we extract the new index of the sorted trials
trial_idx = # your code here

# then, we extract the ordered angles label
angles = # your code here

# apply the same sorting to the neural data and the rates
sorted_neural_data =  # your code here
sorted_rate_neural_data = # your code here

###############################

# Code for plotting

fig = plt.figure(figsize=(12,9), dpi=100)
# heat maps sorted by target angle for three example neurons
for ni,n in enumerate(neurons):
    plt.subplot(3,3,ni+1)
    plt.imshow(sorted_neural_data[:,n], aspect='auto', vmin=0, vmax=1,extent=[time_window[0],time_window[1], 0, len(rate_neural_data)], cmap='magma', origin='lower')
    plt.plot([0,0], [0,len(sorted_neural_data)], 'w--')
    if ni==0: 
        plt.ylabel('trials')
    plt.title(f'neuron {n}, PSTHs')

# heat maps sorted by target angle for three example neurons
for ni,n in enumerate(neurons):
    plt.subplot(3,3,ni+4)
    plt.imshow(sorted_rate_neural_data[:,n], aspect='auto', extent=[time_window[0],time_window[1], 0, len(rate_neural_data)], cmap='magma', origin='lower')
    plt.plot([0,0], [0,len(sorted_rate_neural_data)], 'w--')
    if ni==0: 
        plt.ylabel('trials')
    plt.title(f'neuron {n}, firing rates')

# condition-averaged, scaled & smoothed PSTHs
for ni,n in enumerate(neurons):
    plt.subplot(3,3,ni+7)
    for ai,a in enumerate(targets):
        indeces = (angles == a)
        plt.plot(time_axis, 0.015*ai+np.mean(sorted_rate_neural_data[indeces,n], axis=0), '-', color=t_map[a], zorder=-ai)
        plt.plot([0,0], [-0.05,.4], 'k--')
        if ni==0: 
            plt.ylabel('average activity per target')
        plt.xlim([time_axis[0],time_axis[-1]])
        plt.xlabel('time from movement onset (s)')
        plt.tight_layout()
    plt.ylim(-0.02,0.28)
    plt.title(f'neuron {n}, target-averaged activity')
plt.show()

*  Qualitatively comment on the tuning of these three neurons (you may want to look at other neurons).

## Dimensionality reduction: PCA

The above approaches have a key pitfall: information about the data is lost when averaging, and we fail to find properties of the whole neural population when looking at single neurons. We therefore turn to dimensionality reduction.

As anticipated in **Part I**, PCA is a method that can be applied on matrices. Our dataset is a 3-dimensional array, a tensor, with shape `(n_trials, n_neurons, timesteps)`. For this reason we will need to make it 2-dimensional. This opens to the two types of scenarios we anticipated in section [From Matrices to Tensors](#tensors): neuron-unfolded and trial-unfolded PCA.



### Getting ready


* Before applying PCA, we will first need to rescale our dataset. Although the standard way is to z-score it, we will use here a [min-max normalization](https://en.wikipedia.org/wiki/Feature_scaling), i.e. moving the dataset values to the range [0, 1].

___Apply the min-max normalization for each specific neuron___



In [ ]:
# Fill in your code


* Since we will keep using PCA, we will define our function for computing pca on unfolded data.

In [ ]:
def compute_pca_unfolded(data,n_pcs=None):
    """
    Function computing pca using SVD (np.linalg.svd).
    The last dimension (data.shape[2]) will be the unfolded one.
    If n_pcs is None, n_pcs = all components
       Returns:
       - the first n_pcs principal components of the data 
       - the eigenvector matrix
    """
    # checking the inputs
    assert data.ndim==3, "Data must be 3 dimensional"
    assert isinstance(n_pcs,int) or n_pcs is None, "If provided, n_pcs must be an integer"

    if n_pcs is None:
        n_pcs = data.shape[2]
    
    # Fill in your code
    ################################
        
    # unfold the data. The final shape will be (dim1*dim2, dim3)
    unfolded_data = # your code here

    ################################

    # SVD
    _, _, V = np.linalg.svd(unfolded_data, full_matrices=False)

    # select the first n_pcs components
    if n_pcs is not None:
        V = V[:n_pcs]

    # project the data on the 
    proj_data = data @ V.T

    return proj_data, V, 

### Neural covariability with PCA

**Step 1.** Apply PCA to the neuron-unfolded neural data, using our new function.

In [ ]:
# Fill in your code below

# Let's first check that the dataset has the correct shape
correct_shape = # your code here
flag = norm_rate_neural_data.shape == correct_shape
assert flag, "make sure that norm_rate_neural_data has correct shape."

n_pcs = 20

# apply PCA using the function compute_pca_unfolded
rate_neural_data_scores, V_neuron = # your code here

**Step 2.** Compute and plot cumulative variance explained as a function of the number of principal components. What do you see?

In [ ]:
var_explained = []
for number_of_pcs in range(1,k+1):
    # Fill in your code here
    ##########################
    # let's select the first number_of_pcs principal components from the scores
    sub_scores = # your code here

    # use V_neuron to reconstruct the data
    reconstructed_data = # your code here

    ##########################
    
    # compute and save the variance explained
    mse = ((reconstructed_data - norm_rate_neural_data)**2).mean()
    var_explained.append(1-mse/norm_rate_neural_data.var())

fig = plt.figure(figsize=(4,4), constrained_layout=True)
ax = fig.add_subplot()

ax.plot(np.arange(1, k+1), var_explained, '-o')
ax.set_xlabel('Number of PCs')
ax.set_ylabel('Variance Explained')
ax.set_title('Neuron-unfolded PCA')

plt.show()

**Step 3.**

* Plot the neural activity along the first three PCs in a 3D space. The code was prepared for plotting with three different views, for better visualization.
* Color each trajectory by the target angle.
* You may start by plotting a single trajectory, for only two target angles.
* Finally, increase the plotted trials to 5 and the angles to 3.

In [ ]:
### Some settings

# we extract the angles label
angles = trial_data.angle.values
# targets and color settings
targets = np.sort(trial_data['angle'].unique())
colors = matplotlib.cm.rainbow(np.linspace(0,1,len(targets)))
t_map = {angle: colors[i] for i, angle in enumerate(targets)}

# view settings. You can change them for rotating the plot. (elevation, azimut)
views = [(30, -60),(30,120),(-50,-60)]

# Some parameters you can tune
selected_angles = targets[[0,2,5]]
plotted_trials = 1
principal_components = [0,1,2]


### Plot
fig = plt.figure(figsize=(10,3), constrained_layout=True)
gs = fig.add_gridspec(1, 3)


fig.suptitle(f'Neuron unfolded PCA {np.array(principal_components)+1}')
# looping over views
for k, view in enumerate(views):
    ax = fig.add_subplot(gs[k],projection='3d')
    elev, azim = view
    for a in selected_angles:
        # Fill in your code
        ############################
        
        # create a trial mask by using 'angles' (already done before)
        trial_indeces = # your code here

        # extract the trajectories using trial 
        trajectories = # your code here

        # select only the first plotted_trials-th ones
        trajectories  = # your code here

        # extract the principal components you want to plot
        trajectories = # your code here
        
        ############################
        for trajectory in trajectories:
            ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2],
                color = t_map[a], linewidth = 1.5, alpha=0.8)
    
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel(f'PC{principal_components[0]+1}')
    ax.set_ylabel(f'PC{principal_components[1]+1}')
    ax.set_zlabel(f'PC{principal_components[2]+1}')
    
plt.show()

* What do you observe? not that much, right?



**Step 4** Before giving up, let's try to plot the target-averaged trajectories.
* We will use the same code, but plotting the mean trajectory for each target angle.
* For making the plot more readable, we will also highlight start/end points and the movement onset.
* (Bonus) use `linestyle='o-'` to make the datapoint visible. This allows you to encode the time information. It might be useful to plot every 4/5 timesteps to properly visualize the points
* (Bonus) try to explore different sets of principal components

In [ ]:
### Some settings

# we extract the angles label
angles = trial_data.angle.values
# targets and color settings
targets = np.sort(trial_data['angle'].unique())
colors = matplotlib.cm.rainbow(np.linspace(0,1,len(targets)))
t_map = {angle: colors[i] for i, angle in enumerate(targets)}

# view settings. You can change them for rotating the plot. (elevation, azimut)
views = [(30, -60),(30,120),(-50,-60)]

# Some parameters you can tune
selected_angles = targets
principal_components = [0,1,3]
mov_onset_time = onset_time


### Plot
fig = plt.figure(figsize=(15,5), constrained_layout=True)
gs = fig.add_gridspec(1, 3)


fig.suptitle(f'Neuron unfolded PCA {np.array(principal_components)+1}')
# looping over views
for k, view in enumerate(views):
    ax = fig.add_subplot(gs[k],projection='3d')
    elev, azim = view
    for a in selected_angles:

        # Fill in your code
        ############################

        # create a trial mask by using 'angles' (already done before)
        trial_indeces = # your code here

        # extract the trajectories
        trajectories = # your code here
        
        # compute the mean over trials
        mean_trajectory  = # your code here
        
        # extract the principal components you want to plot
        mean_trajectory = # your code here
        
        # plot mean_trajectory
        ax.plot( # your code here, use the next line settings for better visualization 
            color = t_map[a], markersize=3,alpha=0.8, linewidth=1)
        # plot start points
        ax.scatter( # your code here, use the next line setting for better visualization
                        s=70, edgecolor='k',color = t_map[a],label='start' if a==targets[0] else None)
        # plot mov. onset points
        ax.scatter( # your code here, use the next line setting for better visualization
                        marker = '*', s=70, edgecolor='k',color = t_map[a],label='mov. onset' if a==targets[0] else None)
        # plot stop points
        ax.scatter( # your code here, use the next line setting for better visualization
                        marker = 'v', s=70, edgecolor='k',color = t_map[a],label='stop' if a==targets[0] else None)

    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel(f'PC{principal_components[0]+1}')
    ax.set_ylabel(f'PC{principal_components[1]+1}')
    ax.set_zlabel(f'PC{principal_components[2]+1}')
    if k == 1:
        ax.legend()
    
plt.show()

* OK, we start seeing something! What do you observe? try to connect these plots with the behavioral data.

### Restricting the window

As you already know, PCA is an unsupervised dimensionality reduction method that just try to find axis (aka components) that maximize variance. Although very useful, sometimes it might not be what we want.

Indeed, this might not lead to anything useful if we try to catch some specific information on the task, like target position/direction.

In order to help PCA finding some better representation, let's try to restrict the fitting interval to a time window in which it is more likely to catch the info.

Below, you will find some code that does exactly that. Since it can be confusing, we already prepared the code for you. ___The only thing you need to do is to find the window!___

In [ ]:
def projection_on_time_window(data, start_time, stop_time, n_pcs=4):
    """
    Please enter start/stop times in milliseconds
    """
    assert data.shape[2] == n_neurons, "The data should be in the form ( n_trials, timesteps, n_neurons)"
    start = np.where(time_axis==start_time)[0][0]
    stop = np.where(time_axis==stop_time)[0][0]

    onset_data = norm_rate_neural_data[:,start:stop]
    proj_onset_data, V_onset = compute_pca_unfolded(onset_data, n_pcs)
    proj_full_data = norm_rate_neural_data @ V_onset.T

    angles = trial_data.angle.values
    times = (0,20)

    fig, ax = plt.subplots(1,3,figsize=(13,4), constrained_layout=True)
    for j,pcs in enumerate([(0,1),(1,2),(0,2)]):
        for current_angle in targets:
            trajectory = proj_full_data[angles==current_angle].mean(axis=0)
            ax[j].plot(trajectory[start:stop, pcs[0]], trajectory[start:stop, pcs[1]],'-o',
                    color = t_map[current_angle], linewidth = 1,markersize=2, zorder=1)
            ax[j].scatter(trajectory[stop, pcs[0]], trajectory[stop, pcs[1]],marker='o',
                    color = t_map[current_angle], edgecolor='k',s=120, label='stop' if current_angle==targets[0] else None, zorder=2)
            ax[j].scatter(trajectory[start, pcs[0]], trajectory[start, pcs[1]],marker='v',
                        color = t_map[current_angle], edgecolor='k',s=120,label='start' if current_angle==targets[0] else None, zorder=2)
        ax[j].set_xlabel(f'PC{pcs[0]+1}')
        ax[j].set_ylabel(f'PC{pcs[1]+1}')
        ax[j].set_title(f'Neuron-unfolded PCA {np.array(pcs)+1}')
        if j == 1:
            ax[j].legend()
    plt.show()


    pcs = [0,1,2]
    times = (0,380)
    fig = plt.figure(figsize=(13,4), constrained_layout=True)
    gs = fig.add_gridspec(1, 3)
    fig.suptitle(f'Neuron unfolded PCA {np.array(pcs)+1}')
    for k, view in enumerate([(30, -60),(30,45),(-50,-60)]):
        ax = fig.add_subplot(gs[k],projection='3d')
        elev, azim = view
        for current_angle in targets: 
            trajectory = proj_full_data[angles==current_angle].mean(axis=0)
            ax.plot(trajectory[times[0]:times[1], pcs[0]], trajectory[times[0]:times[1], pcs[1]], trajectory[times[0]:times[1], pcs[2]],'-o',
                    color = t_map[current_angle], linewidth = 1,markersize=2, zorder=1)
            ax.scatter(trajectory[-1, pcs[0]], trajectory[-1, pcs[1]], trajectory[-1, pcs[2]],marker='^',edgecolor='k',
                        color = 'k', linewidth = 1,s=20,label='end' if current_angle==targets[0] else None, zorder=2)
            ax.scatter(trajectory[0, pcs[0]], trajectory[0, pcs[1]], trajectory[0, pcs[2]],marker='v',edgecolor='k',
                        color = 'k', linewidth = 1,s=20,label='start' if current_angle==targets[0] else None, zorder=2)
            ax.scatter(trajectory[stop, pcs[0]], trajectory[stop, pcs[1]], trajectory[stop, pcs[2]],marker='o',edgecolor='k',
                    color = t_map[current_angle],s=80, label='window stop' if current_angle==targets[0] else None, zorder=3)
            ax.scatter(trajectory[start, pcs[0]], trajectory[start, pcs[1]], trajectory[stop, pcs[2]],marker='v',edgecolor='k',
                        color = t_map[current_angle],s=80,label='window start' if current_angle==targets[0] else None, zorder=3)
        ax.view_init(elev=elev, azim=azim)
        ax.set_xlabel(f'PC{pcs[0]+1}')
        ax.set_ylabel(f'PC{pcs[1]+1}')
        ax.set_zlabel(f'PC{pcs[2]+1}')
        if k == 1:
            ax.legend()
    plt.show()

In [ ]:
# Play with the values to see how the projections change

start_time = -900
stop_time = 700

projection_on_time_window(norm_rate_neural_data, start_time, stop_time)

### Trial covariability with PCA

Let's now have a look at trial covariability. 
* Apply PCA to the **trial-unfolded** neural data.

In [ ]:
# Fill in your code below

# Let's first check that the dataset has the correct shape. Use the variables timesteps, n_trials, and n_neurons
correct_shape = ( # your code here ) # tuple of three elements
flag = norm_rate_neural_data.shape == correct_shape
assert flag, "make sure that norm_rate_neural_data has the correct shape."

n_pcs = 20

# apply PCA using the function compute_pca_unfolded
rate_neural_data_scores, V_neuron = # your code here

* Plot cumulative variance explained as a function of the number of principal components.
* Compare this with the equivalent neuron-unfolded plot.

In [ ]:

trial_var_explained = []
for number_of_pcs in range(1,k+1):
    # Fill in your code below
    ###########################

    # let's select the first number_of_pcs principal components from the scores
    sub_scores = # your code here

    # use V_neuron to reconstruct the data
    reconstructed_data = # your code here

    ###########################
    
    # compute and save the variance explained
    mse = ((reconstructed_data - norm_rate_neural_data)**2).mean()
    trial_var_explained.append(1-mse/norm_rate_neural_data.var())

fig = plt.figure(figsize=(4,4), constrained_layout=True)
ax = fig.add_subplot()

ax.plot(np.arange(1, k+1), trial_var_explained, '-o', label='trial-unfolded')
ax.plot(np.arange(1, k+1), neuron_var_explained, '-o', label='neuron-unfolded')
ax.set_xlabel('Number of PCs')
ax.set_ylabel('Variance Explained')
ax.set_title('Neuron-unfolded PCA')
ax.legend()
plt.show()

* What is missing to really assess how much of the data is captured by each unfolding?

**Step 5.** Plot the trial PCs which, as a reminder, are of shape #trials. Additionally sort and color code them by angle.

In [ ]:
pcs_to_plot = 4

fig, axes = plt.subplots(1, pcs_to_plot, figsize=(3*pcs_to_plot,3), constrained_layout=True)

angles = trial_data['angle']
sorted_id = np.argsort(angles)
sorted_angles = angles[sorted_id].values

# We sort by trials to make the plot easier to read
V_trial_sorted = V_trial[:, sorted_id]

for i in range(pcs_to_plot):
    axes[i].scatter(# your code here (leave the rest of the line as it is) , color = [t_map[a] for a in sorted_angles])
    axes[i].set_xlabel('Trial (sorted)')
    axes[i].set_title('PC'+str(i+1))
plt.show()

* What do you observe?

### Bi-cross validation

Let's now try to compute the dimensionality of our data using bi-cross validation. Do you remember how to do it? For make things faster, we did the job for you ;)

In [ ]:
# let's create again our neuron-unfolded data
correct_shape = (n_trials, timesteps, n_neurons)
#norm_rate_neural_data = norm_rate_neural_data.transpose(2,1,0)
flag = norm_rate_neural_data.shape == correct_shape
assert flag, "make sure that norm_rate_neural_data has correct shape."

neuron_unfolded_data = np.concatenate([d for d in norm_rate_neural_data])

neur_indices_tr,neur_indices_te = train_test_split(np.arange(neuron_unfolded_data.shape[1]), train_size = .8, shuffle=True)
samples_indices_tr,samples_indices_te = train_test_split(np.arange(neuron_unfolded_data.shape[0]), train_size = .8, shuffle=True)

# Selecting the training and test data
M_tr_all = neuron_unfolded_data[samples_indices_tr,:]
M_te_all = neuron_unfolded_data[samples_indices_te,:]
M_te_tr = M_te_all[:,neur_indices_tr]
M_te_te = M_te_all[:,neur_indices_te]
# enter here your guess for the rank
your_guess = 8

r_max = 30
dims = np.arange(1,r_max+1)
var_exp_tr = np.zeros((r_max+1))
var_exp_te = np.zeros((r_max+1))

# looping over all different ranks
for r in tqdm(dims):

    # Apply SVD on the training data
    _, S, V = np.linalg.svd(M_tr_all, full_matrices=False)

    # Now, we solve the scores with linear regression. Let's use 'u' for the train basis, 
    # 'z' for the scores and 'x' forthe data
    u = V[:r,neur_indices_tr].T
    x = M_te_tr.T
    # Use linear regression ( u z = x) to solve the scores on test data
    z = np.linalg.lstsq(u,x,rcond=None)[0] # scores

    # Compute the prediction for training neurons
    M_te_tr_predict = (u @ z).T

    # Compute explained variance for te_tr
    mse = ((M_te_tr_predict - M_te_tr)**2).mean()
    var_exp_tr[r] = 1-mse/M_te_tr.var()

    # Compute the prediction on test neurons
    M_te_te_predict = (V[:r,neur_indices_te].T @ z).T

    # Compute variance explained for te_te
    mse = ((M_te_te_predict - M_te_te)**2).mean()
    var_exp_te[r] = 1-mse/M_te_te.var()

fig = plt.figure(figsize=(4,3), constrained_layout=True)
ax = fig.add_subplot()
ax.plot(np.arange(r_max+1),var_exp_tr,'-o', label = "train data")
ax.plot(np.arange(r_max+1),var_exp_te,'-o', label = "test data")
ax.legend()
ax.set_xlabel("Rank")
ax.set_ylabel("Variance explained")
ax.set_ylim([0,1.05])
plt.show()

k_guess = dims[np.argmax(var_exp_te)]

print("Computed rank: ", k_guess)

* What do you observe? 



# **Part V.** Supervised Learning

Until know, we have been focusing on a pure unsupervised methods, i.e. methods that allow to reduce the dimensionality of the dataset without any labeling telling us how good the obtained representation is.<br>

Apart from pure visualization (and fun), our initial goal was to try connecting neural activity with behavioral observation. So, let's now try to use some (simple) supervised learning method for improving our understanding on how PMd/M1 could drive arm trajectories.

 We will fit our model on the neural data and on a PCA projection of the data.

## Some useful functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm

import scipy.ndimage as spnd

def plot_predictions(model, X, y, time_axis, variable = 'position'):
    assert variable in ['position','velocity'], "Please specify 'position' or 'velocity'"
    predictions = np.stack([model.predict(X[i]) for i in range(X.shape[0])])
    fig, axs = plt.subplots(2, 3, figsize=(14, 5))
    trials = X.shape[0]
    colors = matplotlib.cm.rainbow(np.linspace(0,1,trials))
    color_map = {trial_id: colors[i] for i, trial_id in enumerate(range(trials))}
    for i in range(trials):
        # X component
        axs[0][0].plot(time_axis, y[i,:,0], linewidth=0.7, color=color_map[i])
        axs[1][0].plot(time_axis, predictions[i,:,0], linewidth=0.7, color=color_map[i])
        # Y component
        axs[0][1].plot(time_axis, y[i,:,1], linewidth=0.7, color=color_map[i])
        axs[1][1].plot(time_axis, predictions[i,:,1], linewidth=0.7, color=color_map[i])

        # True and predicted trajectories
        if variable == 'velocity':
            true_traj = np.cumsum(y[i], axis=0) * bin_size / 1000
            pred_traj = np.cumsum(predictions[i], axis=0) * bin_size / 1000
        else:
            true_traj = y[i]
            pred_traj = predictions[i]
        axs[0][2].plot(true_traj[:, 0], true_traj[:, 1], linewidth=0.7, color=color_map[i])
        axs[1][2].plot(pred_traj[:, 0], pred_traj[:, 1], linewidth=0.7, color=color_map[i])
    
    # Set up shared axes
    for i in range(2):
        axs[i][0].set_xlim(time_axis[0], time_axis[-1])
        axs[i][1].set_xlim(time_axis[0], time_axis[-1])
        axs[i][2].set_xlim(-180, 180)
        axs[i][2].set_ylim(-140, 140)

    # Add labels
    axs[0][0].set_title(f'X {variable} (mm/s)')
    axs[0][1].set_title(f'Y {variable} (mm/s)')
    axs[0][2].set_title('Reach trajectory')
    axs[0][0].set_ylabel('Target')
    axs[1][0].set_ylabel('Prediction')
    plt.show()

def compute_pca_unfolded(data,n_pcs=None):
    """
    Function computing pca using SVD (np.linalg.svd).
    The last dimension (data.shape[2]) will be the unfolded one.
    If n_pcs is None, n_pcs = all components
       Returns:
       - the first n_pcs principal components of the data 
       - the eigenvector matrix
    """
    assert data.ndim==3, "Data must be 3 dimensional"
    assert isinstance(n_pcs,int) or n_pcs is None, "If provided, n_pcs must be an integer"

    if n_pcs is None:
        n_pcs = data.shape[2]
    
    # unfold the data. The final shape will be (dim1*dim2, dim3)
    unfolded_data = np.concatenate([d for d in data])

    # SVD
    _, _, V = np.linalg.svd(unfolded_data, full_matrices=False)

    # select the first n_pcs components
    if n_pcs is not None:
        V = V[:n_pcs]

    # project the data on the 
    proj_data = data @ V.T

    return proj_data, V

def train_model(x_train, y_train, x_test, y_test, name):

    print("Training '%s'" %name)
    # Fit decoder
    gscv = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 0, 5)})
    gscv.fit(x_train, y_train)

    # evaluating decoder
    train_score = gscv.score(x_train, y_train)
    test_score = gscv.score(x_test, y_test)

    
    print(f" Decoding R2 (train): {train_score}")
    print(f" Decoding R2 (test): {test_score}")

    return gscv.best_estimator_

def run_regression(input_dataset, target_dataset, n_pca_components, time_lag, time_window, selected_target, y_variable = 'position'):

    bin_lag = time_lag//bin_size
    new_time_axis = np.arange(time_window[0],time_window[1],bin_size)
    bin_window = [np.where(time_axis==val)[0][0] for val in time_window]


    targets = np.sort(trial_data['angle'].unique())
    selected_target = targets[3]
    angles = trial_data['angle'].values
    

    # z-scoring the dataset (using standard scaler this time)

    assert input_dataset.shape == (n_trials, timesteps, n_neurons), f"Please check the shape of the dataset. Correct = {(n_trials, timesteps, n_neurons)}"
    input_dataset = np.stack([StandardScaler().fit_transform(input_dataset[:,:,n]) for n in range(n_neurons)],axis=-1)

    # Input datasets (neural data)

    ## Apply time window
    input_dataset = input_dataset[:,bin_window[0]:bin_window[1]]

    ## PCA-transformed dataset
    assert input_dataset.shape == (n_trials, bin_window[1]-bin_window[0], n_neurons), "Please check the shape of the dataset"
    rate_dataset_pca, _ = compute_pca_unfolded(input_dataset, n_pca_components)


    rate_datasets = [input_dataset,rate_dataset_pca]
    rate_datasets_names = ["rate dataset","pca-reduced dataset"]

    # Target datasets

    ## define the lagged window
    window = (bin_window[0] + bin_lag, bin_window[1] + bin_lag)
    ## select correct time window and variables
    targets_dataset = target_dataset[:,:, window[0]:window[1]].transpose(0,2,1)


    # Creating train/test datasets

    train_indeces, test_indeces = train_test_split(np.arange(targets_dataset.shape[0]), train_size = .7, shuffle=True)

    train_rate_datasets = [dataset[train_indeces] for dataset in rate_datasets]
    test_rate_datasets = [dataset[test_indeces] for dataset in rate_datasets]

    y_train = targets_dataset[train_indeces]
    y_test = targets_dataset[test_indeces]

    # making the datasets 2D (samples x n. features)
    train_datasets = [np.concatenate([data for data in dataset]) for dataset in train_rate_datasets]
    test_datasets = [np.concatenate([data for data in dataset]) for dataset in test_rate_datasets]


    y_train = np.concatenate([data for data in y_train])
    y_test = np.concatenate([data for data in y_test])


    # Preparing plot dataset

    # angles_train = trial_data_straight.loc[train_indeces, 'angles'].values
    angles_test = angles[test_indeces]

    # train_index = (angles_train == selected_target)
    test_selection_index = (angles_test == selected_target)

    # train_selection = [dataset[train_index] for dataset in train_rate_datasets]
    test_selection_X = [dataset[test_selection_index] for dataset in test_rate_datasets]
    test_selection_y = targets_dataset[test_indeces][test_selection_index]

    for i, name in enumerate(rate_datasets_names):
        model = train_model(train_datasets[i], y_train, test_datasets[i], y_test, name)

        plot_predictions(model, test_selection_X[i], test_selection_y, new_time_axis, y_variable)

## Hand position decoder

Here, we fit a linear decoder (more precisely, we will perform a Ridge regression) on the neural activity, using it as a decoder for ___predicting hand position___. 

Since coding might take a while, we prepared a function called `run regression`, that implement the regression. What you are asked to do is to set the data and the parameters that are used in the regression:
- `input_dataset`: the neural activity
- `target_dataset`: the behavior dataset (positions)
- `n_pca_components`
- `time_lag`: allows the model to take into account brain-to-movement delays
- `time_window`: selects a particular part of the data for the regression

Try to play with the parameters in order to make the model working better.

In [ ]:
# Tunable parameters
X_dataset = #
y_dataset = #
selected_target = targets[2]
n_pca_components = 3
time_lag = 0 # (ms) Taking into account the delay between cortical activation and the movement
time_window = (-500, 500) # (ms) selecting a time window


# Run the regression
run_regression(X_dataset, y_dataset, n_pca_components, time_lag, time_window, selected_target, 'position')

## Hand velocity

Now, try to do the same for the hand velocity instead, and see if there is any improvement.

In [ ]:
# Tunable parameters
X_dataset = #
y_dataset = #
selected_target = targets[2]
n_pca_components = 3
time_lag = 0 # (ms) Taking into account the delay between cortical activation and the movement
time_window = (-500, 500) # (ms) selecting a time window


# Run the regression
run_regression(X_dataset, y_dataset, n_pca_components, time_lag, time_window, selected_target, 'velocity')


* A little better (maybe). What do you think is happening? Can we do better?

# References

[1] Cunningham, J. P., & Yu, B. M. (2014). Dimensionality reduction for large-scale neural recordings. Nature neuroscience, 17(11), 1500-1509.

[2] Bishop, C. M. (2009). Pattern recognition and machine learning. Springer, New York.

[3] Seely, J. S., Kaufman, M. T., Ryu, S. I., Shenoy, K. V., Cunningham, J. P., & Churchland, M. M. (2016). Tensor analysis reveals distinct population structure that parallels the different computational roles of areas M1 and V1. PLoS computational biology, 12(11), e1005164.

[4] Williams, A. H., Kim, T. H., Wang, F., Vyas, S., Ryu, S. I., Shenoy, K. V., ... & Ganguli, S. (2018). Unsupervised discovery of demixed, low-dimensional neural dynamics across multiple timescales through tensor component analysis. Neuron, 98(6), 1099-1115.

[5] Pellegrino*, A., Stein*, H., & Cayco-Gajic, N. A. (2023). Disentangling mixed classes of covariability in large-scale neural data. bioRxiv.

[6] Owen, A. B., & Perry, P. O. (2009). Bi-cross-validation of the SVD and the nonnegative matrix factorization. Annals of Applied Statistics, 3(2), 564-594.

[7] Mark M. Churchland, John P. Cunningham, Matthew T. Kaufman, Stephen I. Ryu, Krishna V. Shenoy (2010).
Cortical Preparatory Activity: Representation of Movement or First Cog in a Dynamical Machine?. Neuron, Volume 68, Issue 3

[8] Churchland, M., Cunningham, J., Kaufman, M. et al. (2012). Neural population dynamics during reaching. Nature 487, 51–56.
